<a href="https://colab.research.google.com/github/BNarayanaReddy/CS7015/blob/main/Chapter12_SequentialModels_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import torchaudio
import torch.nn as nn
import transformers

In [2]:
class RNN(nn.Module):
    def __init__(self, input_dim,  hidden_dim, output_dim): # output_dim can be vocab size
      super().__init__()
      self.input_dim =input_dim
      self.hidden_size = hidden_dim


      self.hidden_state = nn.Linear(input_dim + hidden_dim, hidden_dim)
      self.output_trans = nn.Linear(input_dim + hidden_dim, output_dim)
      self.sigmoid = nn.Sigmoid()
      self.softmax = nn.Softmax(dim=1)

    def forward(self, x, init_hidden):
      x = torch.cat((x, init_hidden), dim=1)
      h_out = self.hidden_state(x) # h = w_h@(x,h_0) + b
      hidden_state = self.sigmoid(h_out)
      out = self.output_trans(x) # o = w_u@(x,h) + b
      out_probs = self.softmax(out)
      return out_probs, hidden_state

In [3]:
data = "I am an aspiring AI Engineer"

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
emb_model = AutoModel.from_pretrained('google-bert/bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
tokens = tokenizer.tokenize(data)

In [7]:
len(tokens) # vocab size
# tokens

6

In [8]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
# token_ids

In [9]:
# convert to embeddings
token_ids = torch.tensor(token_ids).unsqueeze(dim=0)

In [10]:
token_ids.shape

torch.Size([1, 6])

In [11]:
input_embeddings = emb_model(token_ids)

In [12]:
input_embeddings.last_hidden_state

tensor([[[-0.2779, -0.4092,  0.2379,  ..., -0.0094,  0.3443,  0.1829],
         [-0.0188,  0.1657,  0.2009,  ..., -0.1644,  0.3939,  0.0121],
         [-0.1546,  0.3493,  0.2571,  ..., -0.1441,  0.1436,  0.1813],
         [-0.0513,  0.4066,  0.3496,  ..., -0.1805,  0.2369, -0.3221],
         [-0.1971,  0.2765,  0.3515,  ..., -0.0990,  0.2903, -0.1412],
         [-0.1397,  0.3495,  0.4357,  ..., -0.1079,  0.2744, -0.1135]]],
       grad_fn=<NativeLayerNormBackward0>)

In [13]:
input_embeddings = input_embeddings.last_hidden_state.detach()

In [14]:
input_embeddings.shape

torch.Size([1, 6, 768])

In [15]:
ce_loss = nn.CrossEntropyLoss()

In [16]:
def fit_rnn_model(vocab_size, hidden_dim=256, input_emb_dim = 768, epochs = 10, lr = 1e-4, eps = 1e-8):

  rnn_model = RNN(input_emb_dim, hidden_dim, vocab_size)
  optimizer = torch.optim.Adam(rnn_model.parameters(), lr=lr, eps=eps)

  for epoch in range(1, epochs):
    optimizer.zero_grad()

    # randomly mask the input embeddings
    output_idx = np.random.randint(0, vocab_size)
    input_embeddings[:, output_idx] = torch.zeros(1, input_emb_dim)

    hidden_state = torch.zeros(1, hidden_dim)

    for tensor in input_embeddings[0]:
      # teacher forced training
      output, hidden_state = rnn_model(tensor.unsqueeze(dim=0), hidden_state.detach())

    # After applying forward of all the tokens

    loss = ce_loss(output, torch.tensor(output_idx).unsqueeze(0))
    loss.backward()

    optimizer.step()
    if epoch % 10 == 0:
      print(f"Epoch: {epoch}, Loss: {loss.item()}, Masked Token = {tokens[output_idx]} Output Token: {tokens[torch.argmax(output)]}")

  return output, loss.item()

In [17]:
output, loss = fit_rnn_model(6, epochs = 100)

Epoch: 10, Loss: 1.7509891986846924, Masked Token = engineer Output Token: engineer
Epoch: 20, Loss: 1.81308913230896, Masked Token = ai Output Token: engineer
Epoch: 30, Loss: 1.7556943893432617, Masked Token = engineer Output Token: engineer
Epoch: 40, Loss: 1.762337327003479, Masked Token = engineer Output Token: engineer
Epoch: 50, Loss: 1.792116403579712, Masked Token = am Output Token: engineer
Epoch: 60, Loss: 1.8151408433914185, Masked Token = i Output Token: engineer
Epoch: 70, Loss: 1.818469762802124, Masked Token = i Output Token: engineer
Epoch: 80, Loss: 1.789373517036438, Masked Token = am Output Token: engineer
Epoch: 90, Loss: 1.807407021522522, Masked Token = ai Output Token: engineer
